In [5]:
import pandas as pd
import numpy as np

In [145]:
def load_data(train, test, valid_labels):
    features = set()
    train_m = 0
    test_m = 0

    train_dataset = np.zeros(shape=(1000, 100002))
    test_dataset = np.zeros(shape=(1000, 100002))

    with open(train) as f:
        for i, line in enumerate(f):
            train_m += 1
            
            l = line.strip().split('\t')

            train_dataset[i, 0] = int(l[0])
            # features set for later deletion null
            for x in l[1].split():
                features.add(int(x))
                train_dataset[i ,int(x) + 1] = 1

    test_labels = []
    with open(valid_labels) as f:
        for line in f:
            test_labels.append(int(line.strip()))

    with open(test) as f:
        for i, line in enumerate(f):
            test_m += 1
            
            test_dataset[i, 0] = int(test_labels[i])

            #  features set for later deleting null
            for x in line.strip().split():
                features.add(int(x))
                test_dataset[i ,int(x) + 1] = 1




    # cleaning a dataset a bit
    train_dataset = np.delete(train_dataset,slice(train_m, 1000), 0)
    test_dataset = np.delete(test_dataset,slice(test_m, 1000), 0)

    not_features = {x for x in range(10000)} - features

    not_features = list(map(lambda x: x+1, not_features))
    
    
    train_dataset = np.delete(train_dataset,not_features, 1)
    test_dataset = np.delete(test_dataset,not_features, 1)    

    return train_dataset, test_dataset       


In [149]:
train_data, test_data = load_data('train.dat', 'test.dat', 'valid_labels.dat')

train_x, train_y = train_data[:, 1:], train_data[:, 0]
test_x, test_y = test_data[:, 1:], test_data[:, 0]

test_data[2, 0]



1.0

In [148]:
from sklearn.naive_bayes import GaussianNB


gnb = GaussianNB()

gnb.fit(train_x, train_y)

y_pred = gnb.predict(test_x)

print("1 's in test:", test_y.sum())
print("1 's in predicted:", y_pred.sum())

print("Missmatched :", (y_pred != test_y).sum())

1 's in test: 34.0
1 's in predicted: 0.0
Missmatched : 34


In [133]:

test_labels = []
with open('valid_labels.dat') as f:
    for line in f:
        test_labels.append(int(line.strip()))

sum(test_labels)   

34